In [2]:
import numpy as np
import pandas as pd

In [3]:
book_df=pd.read_csv("F:\\suryakant notes\\Books.csv\\Books.csv",nrows=9850)

In [4]:
book_df.shape

(9850, 8)

In [5]:
book_df.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
dtype: int64

In [6]:
book_df1=book_df.drop(columns=["Image-URL-S","Image-URL-M","Image-URL-L","Year-Of-Publication"])

In [7]:
book_df1["tags"]=book_df1["Book-Author"]+book_df1["Publisher"]

In [8]:
book_df2=book_df1.drop(columns=["Book-Author","Publisher"])

In [9]:
def convertToString(term):
  if type(term) is str:
    return term
  else:
    return str(term)

In [10]:
book_df2["tags"]=book_df2["tags"].apply(convertToString)

In [11]:
book_df2["tags"]=book_df2["tags"].apply(lambda x:x.lower())

In [12]:
book_df2["title"]=book_df2["Book-Title"]

In [13]:
book_df2.drop(columns=["Book-Title"])

,ISBN,tags,title
0,0195153448,mark p. o. morfordoxford university press,Classical Mythology
1,0002005018,richard bruce wrightharperflamingo canada,Clara Callan
2,0060973129,carlo d'esteharperperennial,Decision in Normandy
3,0374157065,gina bari kolatafarrar straus giroux,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,e. j. w. barberw. w. norton &amp; company,The Mummies of Urumchi
...,...,...,...
9845,0061099708,anne rivers siddonsharpertorch,Colony
9846,0425182886,robert b. parkerberkley publishing group,Pot Shot
9847,0425179559,robert b. parkerberkley publishing group,Hugger Mugger
9848,0425174018,robert b. parkerberkley publishing group,Hush Money (Spenser Mysteries)


In [14]:
import nltk

In [15]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [16]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [17]:
book_df2["tags"]=book_df2["tags"].apply(stem)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=10000,stop_words="english")

In [19]:
vectors=cv.fit_transform(book_df2["tags"]).toarray()

In [20]:
cv.get_feature_names()

c:\users\jambhale\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['10',
 '12',
 '16',
 '18',
 '19',
 '2000',
 '62',
 '84',
 'aaron',
 'aarsensteepl',
 'abbeyhenri',
 'abbeysierra',
 'abbi',
 'abbottdov',
 'abbottrandom',
 'abbottshambhala',
 'abelarpenguin',
 'aber',
 'abraham',
 'abrahamplum',
 'abrahamriverhead',
 'abrahamsdoubleday',
 'abram',
 'abzughoughton',
 'acciã³n',
 'achebeanchor',
 'ackermanvintag',
 'ackroydpub',
 'ackroydvintag',
 'ad',
 'adam',
 'adamog',
 'adamsandrew',
 'adamsavon',
 'adamsbaen',
 'adamsballantin',
 'adamsbulfinch',
 'adamsfawcett',
 'adamsharperbusi',
 'adamsharpercollin',
 'adamsheyn',
 'adamskensington',
 'adamsonpantheon',
 'adamsonpenguin',
 'adamspan',
 'adamspocket',
 'adamsrandom',
 'adamvirago',
 'addisbbc',
 'adel',
 'adelin',
 'adkinsperenni',
 'adlerberkley',
 'adlerdel',
 'adlerharpercollin',
 'adlerisland',
 'adlerpenguin',
 'adrian',
 'adriana',
 'adriennetim',
 'adult',
 'africa',
 'ag',
 'agatha',
 'agn',
 'aguiarediã',
 'agustã',
 'agutcolumna',
 'ahdaf',
 'ahlbergpenguin',
 'ahnekiepenheu',
 'ai',

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
similarity=cosine_similarity(vectors)

In [23]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1581, 0.75),
 (545, 0.6708203932499369),
 (817, 0.5773502691896258),
 (986, 0.5773502691896258),
 (1798, 0.5773502691896258)]

In [24]:
def recommend(book):
    book_index=book_df2[book_df2["title"]==book].index[0]
    distance=similarity[book_index]
    book_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    for i in book_list:
        print(book_df2.iloc[i[0]].title)

In [25]:
recommend("Colony")

Hill Towns
Colony
Hill Towns
Up Island
Peachtree Road


In [26]:
import pickle

In [27]:
pickle.dump(book_df2.to_dict(),open("book.pkl","wb"))

In [28]:
pickle.dump(similarity,open("similarity.pkl","wb"))